In [1]:
# installing the required libraries
!pip install langchain-community==0.2.4 langchain==0.2.3 faiss-cpu==1.8.0 unstructured==0.14.5 unstructured[pdf]==0.14.5 transformers==4.41.2 sentence-transformers==3.0.1

In [2]:
# loading the document
from langchain_community.document_loaders import UnstructuredFileLoader

file_path = "C:/Users/othma/Bureau/My project/3.5. Contrat d_inscription - M2, SAMIH, Othman - signed (2).pdf"
loader = UnstructuredFileLoader(file_path)
data = loader.load()

In [3]:
# spliting the document into chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(data)

In [4]:
# loading the embeddings model
from langchain_huggingface import HuggingFaceEmbeddings

embeddings_model = HuggingFaceEmbeddings()

C:\Users\othma\anaconda3\envs\venv1\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
C:\Users\othma\anaconda3\envs\venv1\Lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
# storing the embeddings in the vector database
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(all_splits, embeddings_model)

In [6]:
# retreiving the relevant chunks embeddings
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [7]:
# loading the Gemma2 model
from langchain_ollama.llms import OllamaLLM

llm = OllamaLLM(model="gemma2:2b")

In [8]:
# getting the model's response
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [15]:
response = rag_chain.invoke({"input": "En cas de refus de visa?"})
print(response["answer"])

If you are denied a student visa, the non-refundable fees will not be reimbursed, but you have options to follow e-learning courses, delay your integration, or apply for sponsorship from another eligible person.  

